<a href="https://colab.research.google.com/github/ggggg-15/DataAnalytics/blob/main/DA_Asgn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install --upgrade gspread pandas gspread-dataframe


In [16]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from gspread_dataframe import get_as_dataframe
import numpy as np
import pandas as pd
from collections import defaultdict
import math

# Authenticate and initialize the Google Sheets API
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet using the URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1k4LkeZctJlmkzwnePxfaG3hfD-pgsJ8pFH5clW9FlqA/edit?usp=sharing'

sheet = gc.open_by_url(sheet_url)

# Select the specific worksheet within the sheet
worksheet = sheet.get_worksheet(0)  # Assuming you want the first sheet

# Convert the worksheet into a Pandas DataFrame
df = get_as_dataframe(worksheet)

# Display the DataFrame
print(df.head())


    age         workclass  education  educationno       maritalstatus  \
0  39.0         State-gov  Bachelors         13.0       Never-married   
1  50.0  Self-emp-not-inc  Bachelors         13.0  Married-civ-spouse   
2  38.0           Private    HS-grad          9.0            Divorced   
3  53.0           Private       11th          7.0  Married-civ-spouse   
4  28.0           Private  Bachelors         13.0  Married-civ-spouse   

          occupation   relationship   race     sex  capitalgain  capitalloss  \
0       Adm-clerical  Not-in-family  White    Male       2174.0          0.0   
1    Exec-managerial        Husband  White    Male          0.0          0.0   
2  Handlers-cleaners  Not-in-family  White    Male          0.0          0.0   
3  Handlers-cleaners        Husband  Black    Male          0.0          0.0   
4     Prof-specialty           Wife  Black  Female          0.0          0.0   

   hoursperweek         native Possibility  
0          40.0  United-States     

In [17]:
# Check for missing values
print("Missing values in each column before handling:")
print(df.isnull().sum())

# Handle missing numerical values (e.g., age, capitalgain, hoursperweek) by filling with the mean
numerical_cols = ['age', 'capitalgain', 'capitalloss', 'hoursperweek']
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)

# Handle missing categorical values (e.g., workclass, occupation, native) by filling with the mode
categorical_cols = ['workclass', 'occupation', 'native']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Check for missing values after handling
print("\nMissing values in each column after handling:")
print(df.isnull().sum())

# Display the first few rows of the cleaned DataFrame
cleaned_df = df.head()
print("\nFirst few rows of the cleaned dataset:")
print(cleaned_df)

# Save the cleaned DataFrame to an Excel file
df.to_excel('cleaned_dataset.xlsx', index=False)

Missing values in each column before handling:
age                0
workclass          0
education          0
educationno        0
maritalstatus    200
occupation         0
relationship       0
race             681
sex              342
capitalgain        0
capitalloss        0
hoursperweek     466
native             0
Possibility        0
dtype: int64

Missing values in each column after handling:
age                0
workclass          0
education          0
educationno        0
maritalstatus    200
occupation         0
relationship       0
race             681
sex              342
capitalgain        0
capitalloss        0
hoursperweek       0
native             0
Possibility        0
dtype: int64

First few rows of the cleaned dataset:
    age         workclass  education  educationno       maritalstatus  \
0  39.0         State-gov  Bachelors         13.0       Never-married   
1  50.0  Self-emp-not-inc  Bachelors         13.0  Married-civ-spouse   
2  38.0           Private    HS-g

<ipython-input-17-2013a56ba99f>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
<ipython-input-17-2013a56ba99f>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [18]:
# Convert categorical columns to numerical values
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

X = df.drop(columns='Possibility')
y = df['Possibility'].map({'>0.5': 'criminal', '<=0.5': 'innocent'})

X = X.apply(pd.to_numeric, errors='coerce')
X.fillna(0, inplace=True)  # Replace NaNs with 0 if there are any

# Train-test split (simple split for demonstration)
train_size = int(0.8 * len(df)) ############## can change ratio to change or random selection for best accuracy
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(X_train.dtypes)

age                       float64
education                 float64
educationno               float64
maritalstatus             float64
relationship              float64
                           ...   
native_Thailand              bool
native_Trinadad&Tobago       bool
native_United-States         bool
native_Vietnam               bool
native_Yugoslavia            bool
Length: 68, dtype: object


In [19]:
# Implement Naive Bayes classifier
class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = {}
        self.feature_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
        self.features = []

    def fit(self, X, y):
        self.features = X.columns

        # Calculate class probabilities
        class_counts = y.value_counts() # number of classes : criminal and innocent
        total_samples = len(y) # num of rows
        self.class_probs = {cls: count / total_samples for cls, count in class_counts.items()}

        # Calculate feature probabilities
        for feature in self.features:
            for cls in self.class_probs:
                class_df = X[y == cls]
                feature_values = class_df[feature].values
                mean = np.mean(feature_values)
                std = np.std(feature_values)
                self.feature_probs[feature][cls]['mean'] = mean
                self.feature_probs[feature][cls]['std'] = std

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            class_probs = {}
            for cls in self.class_probs:
                prob = np.log(self.class_probs[cls])
                for feature in self.features:
                    mean = self.feature_probs[feature][cls]['mean']
                    std = self.feature_probs[feature][cls]['std']
                    x = row[feature]
                    prob += -0.5 * np.log(2 * np.pi * std ** 2) - (x - mean) ** 2 / (2 * std ** 2)
                class_probs[cls] = prob
            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)
        return np.array(predictions)

In [20]:
import numpy as np

class NB:
    def __init__(self):
        self.class_probs = {} ### prior prob
        self.feature_probs = {} ### conditional prob

    def fit(self, X, y):
        total_samples = len(y)
        self.classes = np.unique(y)

        # Calculate class probabilities (priors)
        for cls in self.classes:
            self.class_probs[cls] = np.sum(y == cls) / total_samples

        # Calculate feature probabilities (likelihoods)
        for feature in X.columns:
            self.feature_probs[feature] = {}
            for cls in self.classes:
                class_data = X[y == cls]
                self.feature_probs[feature][cls] = class_data[feature].value_counts(normalize=True).to_dict()

    def predict(self, X):
        predictions = []
        for i, row in X.iterrows():
            class_probs = {}
            for cls in self.classes:
                class_probs[cls] = np.log(self.class_probs[cls])  # Log of the prior probability
                for feature in X.columns:
                    feature_value = row[feature]
                    if feature_value in self.feature_probs[feature][cls]:
                        class_probs[cls] += np.log(self.feature_probs[feature][cls][feature_value])
                    else:
                        class_probs[cls] += np.log(1e-6)  # Smoothing for unseen feature values

            predictions.append(max(class_probs, key=class_probs.get))
        return predictions


In [21]:
# Train Naive Bayes model
# nb_classifier = NaiveBayesClassifier()
nb_classifier = NB()
nb_classifier.fit(X_train, y_train)

# Make predictions
y_pred = nb_classifier.predict(X_test)

# Evaluate accuracy
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.82


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# NAIVE BAYES (by sklearn)

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

# Evaluate
nb_accuracy = accuracy_score(y_test, y_pred_nb)
print("Sklearn Naive Bayes Accuracy: {:.2f}".format(nb_accuracy))
print("Classification Report:\n", classification_report(y_test, y_pred_nb))

Sklearn Naive Bayes Accuracy: 0.80
Classification Report:
               precision    recall  f1-score   support

    criminal       0.68      0.39      0.49      1533
    innocent       0.82      0.94      0.87      4500

    accuracy                           0.80      6033
   macro avg       0.75      0.66      0.68      6033
weighted avg       0.78      0.80      0.78      6033



In [23]:
# Support Vector Machine (SVM)
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy: {:.2f}".format(svm_accuracy))
print("Classification Report:\n", classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.80
Classification Report:
               precision    recall  f1-score   support

    criminal       0.76      0.28      0.41      1533
    innocent       0.80      0.97      0.88      4500

    accuracy                           0.80      6033
   macro avg       0.78      0.63      0.64      6033
weighted avg       0.79      0.80      0.76      6033



In [30]:
# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

# Evaluate
dt_accuracy = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy: {:.2f}".format(dt_accuracy))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))


Decision Tree Accuracy: 0.78
Classification Report:
               precision    recall  f1-score   support

    criminal       0.57      0.59      0.58      1533
    innocent       0.86      0.85      0.85      4500

    accuracy                           0.78      6033
   macro avg       0.71      0.72      0.72      6033
weighted avg       0.79      0.78      0.78      6033



In [25]:
# K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy: {:.2f}".format(knn_accuracy))
print("Classification Report:\n", classification_report(y_test, y_pred_knn))


KNN Accuracy: 0.82
Classification Report:
               precision    recall  f1-score   support

    criminal       0.68      0.56      0.61      1533
    innocent       0.86      0.91      0.88      4500

    accuracy                           0.82      6033
   macro avg       0.77      0.74      0.75      6033
weighted avg       0.81      0.82      0.82      6033



In [26]:
# Comparison summary
print("\n--- Model Comparison ---")
print("Custom Naive Bayes Accuracy:", accuracy)
print("Sklearn Naive Bayes Accuracy:", nb_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)
print("KNN Accuracy:", knn_accuracy)


--- Model Comparison ---
Custom Naive Bayes Accuracy: 0.824133930051384
Sklearn Naive Bayes Accuracy: 0.7984419028675618
SVM Accuracy: 0.7952925575998674
Decision Tree Accuracy: 0.7838554616277142
KNN Accuracy: 0.8214818498259573


In [27]:
import numpy as np
from collections import Counter

class EnsembleClassifier:
    def __init__(self, models):
        """
        Initialize the ensemble with a list of models.
        :param models: A list of tuples, where each tuple contains the model name and the model itself.
        Example: [('Custom NB', custom_nb_classifier), ('SVM', svm_model), ...]
        """
        self.models = models

    def fit(self, X_train, y_train):
        """
        Fit all models on the training data.
        :param X_train: Training features
        :param y_train: Training labels
        """
        for name, model in self.models:
            model.fit(X_train, y_train)

    def predict(self, X_test):
        """
        Predict using majority voting.
        :param X_test: Test features
        :return: Final predictions based on majority vote
        """
        predictions = []

        # Get predictions from all models
        for name, model in self.models:
            predictions.append(model.predict(X_test))

        # Transpose the list of predictions to get a list of predictions for each instance
        predictions = np.array(predictions).T

        # Perform majority voting
        final_predictions = []
        for instance_preds in predictions:
            # Majority vote (most common prediction)
            final_predictions.append(Counter(instance_preds).most_common(1)[0][0])

        return np.array(final_predictions)


In [28]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# Instantiate your custom Naive Bayes model
# custom_nb_classifier = NaiveBayesClassifier()
custom_nb_classifier = NB()

# Instantiate sklearn models
svm_model = SVC()
dt_model = DecisionTreeClassifier()
knn_model = KNeighborsClassifier()

# List of models for the ensemble
models = [
    ('Custom NB', custom_nb_classifier),
    ('SVM', svm_model),
    ('Decision Tree', dt_model),
    ('KNN', knn_model)
]

# Instantiate the ensemble classifier
ensemble_classifier = EnsembleClassifier(models=models)

# Fit the ensemble model
ensemble_classifier.fit(X_train, y_train)

# Predict using the ensemble model
y_pred_ensemble = ensemble_classifier.predict(X_test)

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_test, y_pred_ensemble)
ensemble_f1_score = f1_score(y_test, y_pred_ensemble, average='weighted')

print("Ensemble Model Accuracy: {:.2f}".format(ensemble_accuracy))
print("Ensemble Model F1 Score: {:.2f}".format(ensemble_f1_score))

# Optional: Print confusion matrix and classification report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_ensemble))
print("Classification Report:\n", classification_report(y_test, y_pred_ensemble))


Ensemble Model Accuracy: 0.84
Ensemble Model F1 Score: 0.83
Confusion Matrix:
 [[ 865  668]
 [ 311 4189]]
Classification Report:
               precision    recall  f1-score   support

    criminal       0.74      0.56      0.64      1533
    innocent       0.86      0.93      0.90      4500

    accuracy                           0.84      6033
   macro avg       0.80      0.75      0.77      6033
weighted avg       0.83      0.84      0.83      6033



In [29]:
import time

# Measure time for training the ensemble
start_time = time.time()
ensemble_classifier.fit(X_train, y_train)
training_time = time.time() - start_time
print(f"Ensemble Model Training Time: {training_time:.2f} seconds")

# Measure time for making predictions
start_time = time.time()
y_pred_ensemble = ensemble_classifier.predict(X_test)
prediction_time = time.time() - start_time
print(f"Ensemble Model Prediction Time: {prediction_time:.2f} seconds")


Ensemble Model Training Time: 46.50 seconds
Ensemble Model Prediction Time: 12.06 seconds
